In [2]:
# variables
T_c = 0.00000009775
f_L1 = 1575420000
N_i = 4
CN0 = 15
B_pll = 0.5
B_fe = 26000000
T = 0.020
d = 0.3
pi = math.pi
c = 299792458
B_dll = 0.5
dt = 7
sigma_uere = 0 # can't find a value for this

NameError: name 'math' is not defined

In [ ]:
#Values from random table in Source 31
# h_0 = 1.4*(10**(-22))
# h_m1 = 2.3*(10**(-26))
# h_m2 = 3.3*(10**(-31))


#Values from sample clocks in Source 30
h_0 = 1.8*(10**(-21))
h_m1 = 6.492*(10**(-22))
h_m2 = 1.37*(10**(-24))

pi = math.pi

e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*h_m2*(pi**2)*(dt**3)
e2 = 2*h_m1*dt + h_m2*((pi*dt)**2)
e3 = ((h_0/(2*dt))+ 2*h_m1 + (8/3)*(pi*pi)*h_m2*dt)

Q = (c**2)*np.array([[e1,e2],[e2,e3]])
print(Q)

[[0.00656212 0.00087641]
 [0.00087641 0.00015093]]


In [1]:
s_dll = (B_dll/(2*CN0))*(1/(T_c*B_fe))*(1+(1/(T*CN0)))
pr_var = ((c*T_c)**2)*s_dll+ sigma_uere

s_pll = (B_pll/(CN0))*(1+(1/(2*T*CN0)))
prr_var = ((c**2)/((2*pi*f_L1 * T_c)**2)) * (s_pll)


R = lambda N : np.kron(np.eye(N),np.array([[pr_var,0],[0,prr_var]]))

print(R(2))

NameError: name 'B_dll' is not defined

In [ ]:
R_cf = lambda num_msmts: np.kron(np.eye(sum(num_msmts)),R(1))

measure_noise = lambda num_msmts : np.linalg.cholesky(R_cf(num_msmts)) @ np.random.normal(np.array([[0 for _ in range(2*sum(num_msmts))]]).T)

In [ ]:
sigma_j = 10**(-16)

In [ ]:
def run_both_filters(iterations, num_msmts,C,F_full,stations,kf, x, convergence = None):
    n = len(stations)
    C_ckf = (1/n)*np.ones((n,n))
    Q_full = np.kron(np.eye(n),Q)
    #Process Noise
    procc_noise = lambda : np.linalg.cholesky(Q_full) @ np.random.normal(
                                                        np.array([[0 for _ in range(2*n)]]).T)
    def get_station_truth(x,id):
        return np.array([[x[2*id][0]],[x[2*id+1][0]]])

    truth = np.zeros((iterations,2*n,1))
    measurements = []

    raw_pred_df = np.zeros((iterations,n,2,1))
    predictions_df = np.zeros((iterations,n,2,1))
    updated_pred = np.zeros((iterations,n,2,1))
    errors_df = np.zeros((iterations,n,2,1))
    P_hist_df = np.zeros((iterations,n,2,2))

    predictions_cf = [x]
    errors_cf = []
    P_hist_cf = []
    #Process Noise

    T_g = 0

    for i in range(iterations):
        #This is our new truth

        x = F_full@x + procc_noise()
        truth[i] = x

        T_g += dt
        true_biases = [x[::2][i]/c for i in range(n)]
        true_drifts = [x[1::2][i]/c for i in range(n)]
        z = (H_cf(num_msmts[i]) @ truth[i]) + measure_noise(num_msmts[i])
        gps_measurements = [z[(2*sum(num_msmts[i][:k])):(2*sum(num_msmts[i][:k+1]))] for k in range(n)]
        measurements.append(z)

        kf.predict()
        kf.msmt_sharing(T_g,true_biases)
        if sum(num_msmts[i]) > 0:
            kf.update(z,num_msmts[i]) # Correct estimate using measurement

        predictions_cf.append(kf.x) # Record updated estimate and error
        errors_cf.append(kf.x - truth[i])

        P_hist_cf.append(kf.P) # Append the covariance matrix history


        if i == 0:
            time_x,gps_x,time_ests, cov_ests = diffusion_filter_iteration(
                stations,Q,R,C,gps_measurements,None,None,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])
        else:
            time_x,gps_x,time_ests, cov_ests = diffusion_filter_iteration(
            stations,Q,R,C,gps_measurements,time_ests,cov_ests,true_biases,true_drifts,T_g,dt,sigma_j,N=num_msmts[i])
        errors_df[i] = np.array([station.x - get_station_truth(x,station.id) for station in stations])
        raw_pred_df[i] = np.array(time_x)
        updated_pred[i] = np.array(gps_x)
        predictions_df[i] = np.array([stn.x for stn in stations])
        
        P_hist_df[i] = np.array([station.P for station in stations])
    
    return errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements,predictions_cf,predictions_df
